## Query Synapses

In [130]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np

In [131]:
url = "https://global.brain-wire-test.org/"
datastack = "h01_c3_flat"

In [132]:
client = CAVEclient(datastack_name=datastack, server_address=url)

In [133]:
datastack = client.info.get_datastack_info('h01_c3_flat')
datastack['synapse_table']

'synapses_using_sample_voxel_coords'

## Set version of materailized database

In [15]:
client.materialize.get_versions()

[406, 415, 418, 419, 420]

In [16]:
client.materialize.version

np.int64(420)

In [17]:
# get all available tables
client.materialize.get_tables()

['cells',
 'nucleus',
 'synapses',
 'proofreading_status_test',
 'synapses_using_sample_voxel_coords']

## Scenario

1. Find a cell that interests you to explore. For example, you could filter by cell type, "Interneuron."
2. Find synapses in which pre/post-synaptic segment ID is matched with the `pt_root_id` of the cell.
3. In our current datastack, the `synapses_using_sample_voxel_coords` table's `pre_pt_position` and `post_pt_position` are not coordinates of the synaptic point; they can indicate the coordinate of pre/post-neuron.
4. Find all of the synapses using `ctr_pt_position` in the `synapses` table since `ctr_pt_position` is a joint column between the `synapses_using_sample_voxel_coords` table and the `synapses` table. The `ctr_pt_position` is a centroid of the synapse bounding box.

## Find a Cell by Cell type

In [18]:
cells = client.materialize.query_table("cells", 
                                       filter_equal_dict = {"cell_type": "INTERNEURON"})

In [19]:
cells.iloc[0]

id                                                  23557
created                  2023-06-15 19:28:30.336583+00:00
superceded_id                                         NaN
valid                                                   t
classification_system                             0, 0, 0
cell_type                                     INTERNEURON
pt_supervoxel_id                       124865765814829056
pt_root_id                             864691132192363248
pt_position                        [384127, 235982, 5120]
Name: 0, dtype: object

## Find Synapses Corresponding to the Cell

In [20]:
# show synapse examples in synapses_using_sample_voxel_coords table
client.materialize.query_table("synapses_using_sample_voxel_coords", limit=3)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40971523,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,134364238863925557,864691128621846602,134293870119747633,864691131861340864,"[453256, 225950, 1879]","[453086, 226004, 1838]","[453312, 225966, 1883]"
1,40971525,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,122744257927708975,864691132202350435,123448011605934094,864691132349699414,"[369038, 157894, 4777]","[373770, 158410, 12]","[368610, 158562, 4703]"
2,40971526,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,106917542483722336,864691132270145118,106776805062476142,864691132270173278,"[253634, 204356, 1947]","[252889, 204590, 2104]","[253668, 204394, 1970]"


In [48]:
synapses_sample_voxel_df = client.materialize.query_table("synapses_using_sample_voxel_coords",
                                          filter_equal_dict = {"post_pt_root_id": cells.iloc[0].pt_root_id},
                                        )

In [51]:
synapses_sample_voxel_df

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,124732629,2024-04-12 03:04:21.160222+00:00,NaN,t,NaN,125077834052927520,864691131402032897,124795603161973333,864691132192363248,"[385654, 242892, 4992]","[383760, 237104, 5089]","[385690, 242925, 5004]"
1,124793378,2024-04-12 03:04:23.137601+00:00,NaN,t,NaN,124795603296190471,864691131338669220,124795603161973333,864691132192363248,"[383636, 237104, 5250]","[383760, 237104, 5089]","[383533, 236885, 5225]"
2,124830931,2024-04-12 03:04:24.612698+00:00,NaN,t,NaN,124725096978841972,864691128832265908,124795603161973333,864691132192363248,"[383236, 236080, 5097]","[383760, 237104, 5089]","[383298, 236203, 5071]"
3,124799332,2024-04-12 03:04:23.399908+00:00,NaN,t,NaN,125007533961118293,864691132154403015,124795603161973333,864691132192363248,"[385458, 243310, 4987]","[383760, 237104, 5089]","[386256, 244336, 5171]"
4,124827681,2024-04-12 03:04:24.612698+00:00,NaN,t,NaN,124725028326473803,864691129350122755,124795603161973333,864691132192363248,"[383318, 235571, 5137]","[383760, 237104, 5089]","[383326, 235739, 5186]"
...,...,...,...,...,...,...,...,...,...,...,...,...
57,124817986,2024-04-12 03:04:24.327687+00:00,NaN,t,NaN,124936340650328539,864691132036927803,124795603161973333,864691132192363248,"[384690, 237104, 5020]","[383760, 237104, 5089]","[384754, 236994, 5065]"
58,124842364,2024-04-12 03:04:24.880945+00:00,NaN,t,NaN,124936340650328515,864691132036928571,124795603161973333,864691132192363248,"[384618, 237114, 5019]","[383760, 237104, 5089]","[384809, 237250, 5139]"
59,124843135,2024-04-12 03:04:24.880945+00:00,NaN,t,NaN,124936271930851452,864691128468033067,124795603161973333,864691132192363248,"[384570, 236970, 4992]","[383760, 237104, 5089]","[384549, 236980, 5005]"
60,124799264,2024-04-12 03:04:23.399908+00:00,NaN,t,NaN,125007533961118293,864691132154403015,124795603161973333,864691132192363248,"[385458, 243310, 4987]","[383760, 237104, 5089]","[386030, 243792, 5139]"


In [28]:
synapse_table = client.info.get_datastack_info('h01_c3_flat')["synapse_table"]
all_synapses = []

def get_synapse(ctr_pt_position):
    diff = 5
    bounding_box = [ctr_pt_position, ctr_pt_position + diff]
    synapse = client.materialize.query_table(
        synapse_table,
        filter_spatial_dict={"ctr_pt_position": bounding_box},
        select_columns = ["pre_pt_position", "post_pt_position", "ctr_pt_position"]
    )
    return synapse

for idx, row in synapses_positions.iterrows():
    ctr_pt_position = row['ctr_pt_position']
    synapse = get_synapse(ctr_pt_position)
    all_synapses.append(synapse)
    
all_synapses_df = pd.concat(all_synapses, ignore_index=True)
all_synapses_df

,pre_pt_position,post_pt_position,ctr_pt_position
0,"[385684, 242927, 5004]","[385696, 242924, 5004]","[385690, 242925, 5004]"
1,"[383527, 236888, 5225]","[383538, 236885, 5225]","[383533, 236885, 5225]"
2,"[383293, 236204, 5071]","[383305, 236202, 5071]","[383298, 236203, 5071]"
3,"[386256, 244339, 5170]","[386256, 244336, 5171]","[386256, 244336, 5171]"
4,"[383320, 235736, 5186]","[383331, 235741, 5186]","[383326, 235739, 5186]"
...,...,...,...
57,"[384759, 236995, 5065]","[384747, 236992, 5065]","[384754, 236994, 5065]"
58,"[384815, 237250, 5139]","[384803, 237249, 5139]","[384809, 237250, 5139]"
59,"[384552, 236981, 5005]","[384542, 236977, 5007]","[384549, 236980, 5005]"
60,"[386022, 243804, 5140]","[386036, 243790, 5139]","[386030, 243792, 5139]"


In [52]:
synapses_sample_voxel_df = synapses_sample_voxel_df.drop(columns=['pre_pt_position', 'post_pt_position'])
synapses_sample_voxel_df['tuple_key'] = synapses_sample_voxel_df['ctr_pt_position'].apply(tuple)
all_synapses_df['tuple_key'] = all_synapses_df['ctr_pt_position'].apply(tuple)

In [57]:
merged_synapses_df = pd.merge(synapses_sample_voxel_df, all_synapses_df, on='tuple_key', how='inner')

In [58]:
merged_synapses_df

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,ctr_pt_position_x,tuple_key,pre_pt_position,post_pt_position,ctr_pt_position_y
0,124732629,2024-04-12 03:04:21.160222+00:00,NaN,t,NaN,125077834052927520,864691131402032897,124795603161973333,864691132192363248,"[385690, 242925, 5004]","(385690, 242925, 5004)","[385684, 242927, 5004]","[385696, 242924, 5004]","[385690, 242925, 5004]"
1,124793378,2024-04-12 03:04:23.137601+00:00,NaN,t,NaN,124795603296190471,864691131338669220,124795603161973333,864691132192363248,"[383533, 236885, 5225]","(383533, 236885, 5225)","[383527, 236888, 5225]","[383538, 236885, 5225]","[383533, 236885, 5225]"
2,124830931,2024-04-12 03:04:24.612698+00:00,NaN,t,NaN,124725096978841972,864691128832265908,124795603161973333,864691132192363248,"[383298, 236203, 5071]","(383298, 236203, 5071)","[383293, 236204, 5071]","[383305, 236202, 5071]","[383298, 236203, 5071]"
3,124799332,2024-04-12 03:04:23.399908+00:00,NaN,t,NaN,125007533961118293,864691132154403015,124795603161973333,864691132192363248,"[386256, 244336, 5171]","(386256, 244336, 5171)","[386256, 244339, 5170]","[386256, 244336, 5171]","[386256, 244336, 5171]"
4,124827681,2024-04-12 03:04:24.612698+00:00,NaN,t,NaN,124725028326473803,864691129350122755,124795603161973333,864691132192363248,"[383326, 235739, 5186]","(383326, 235739, 5186)","[383320, 235736, 5186]","[383331, 235741, 5186]","[383326, 235739, 5186]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,124817986,2024-04-12 03:04:24.327687+00:00,NaN,t,NaN,124936340650328539,864691132036927803,124795603161973333,864691132192363248,"[384754, 236994, 5065]","(384754, 236994, 5065)","[384759, 236995, 5065]","[384747, 236992, 5065]","[384754, 236994, 5065]"
58,124842364,2024-04-12 03:04:24.880945+00:00,NaN,t,NaN,124936340650328515,864691132036928571,124795603161973333,864691132192363248,"[384809, 237250, 5139]","(384809, 237250, 5139)","[384815, 237250, 5139]","[384803, 237249, 5139]","[384809, 237250, 5139]"
59,124843135,2024-04-12 03:04:24.880945+00:00,NaN,t,NaN,124936271930851452,864691128468033067,124795603161973333,864691132192363248,"[384549, 236980, 5005]","(384549, 236980, 5005)","[384552, 236981, 5005]","[384542, 236977, 5007]","[384549, 236980, 5005]"
60,124799264,2024-04-12 03:04:23.399908+00:00,NaN,t,NaN,125007533961118293,864691132154403015,124795603161973333,864691132192363248,"[386030, 243792, 5139]","(386030, 243792, 5139)","[386022, 243804, 5140]","[386036, 243790, 5139]","[386030, 243792, 5139]"


In [121]:
client.materialize.query_table("synapses", limit=3)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,0,2023-07-01 14:55:35.673208+00:00,NaN,t,NaN,1,1,1,1,"[270395, 161567, 256]","[270389, 161574, 256]","[270393, 161571, 256]"
1,1,2023-07-01 14:55:35.673208+00:00,NaN,t,NaN,1,1,1,1,"[290329, 114538, 256]","[290337, 114546, 256]","[290332, 114543, 256]"
2,2,2023-07-01 14:55:35.673208+00:00,NaN,t,NaN,1,1,1,1,"[391940, 69574, 256]","[391951, 69573, 256]","[391946, 69574, 256]"


In [125]:
client.materialize.query_table("synapses_using_sample_voxel_coords", limit=3)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40971523,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,134364238863925557,864691128621846602,134293870119747633,864691131861340864,"[453256, 225950, 1879]","[453086, 226004, 1838]","[453312, 225966, 1883]"
1,40971525,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,122744257927708975,864691132202350435,123448011605934094,864691132349699414,"[369038, 157894, 4777]","[373770, 158410, 12]","[368610, 158562, 4703]"
2,40971526,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,106917542483722336,864691132270145118,106776805062476142,864691132270173278,"[253634, 204356, 1947]","[252889, 204590, 2104]","[253668, 204394, 1970]"


## Synapse Query

In [135]:
client.info.get_datastack_info('h01_c3_flat')['synapse_table']

'synapses_using_sample_voxel_coords'

In [137]:
client.materialize.query_table("synapses_using_sample_voxel_coords", limit=3)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40971523,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,134364238863925557,864691128621846602,134293870119747633,864691131861340864,"[453256, 225950, 1879]","[453086, 226004, 1838]","[453312, 225966, 1883]"
1,40971525,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,122744257927708975,864691132202350435,123448011605934094,864691132349699414,"[369038, 157894, 4777]","[373770, 158410, 12]","[368610, 158562, 4703]"
2,40971526,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,106917542483722336,864691132270145118,106776805062476142,864691132270173278,"[253634, 204356, 1947]","[252889, 204590, 2104]","[253668, 204394, 1970]"


In [136]:
client.materialize.synapse_query(post_ids=[864691131861340864, 864691132349699414, 864691132270173278])

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40869646,2024-04-12 00:14:36.782540+00:00,NaN,t,NaN,123942379588682010,864691132400996338,123448011605934094,864691132349699414,"[377628, 171754, 223]","[373770, 158410, 12]","[373974, 173811, 799]"
1,40960264,2024-04-12 00:14:39.879227+00:00,NaN,t,NaN,122043248733585629,864691131912892749,123448011605934094,864691132349699414,"[363756, 178168, 1305]","[373770, 158410, 12]","[363821, 178189, 1339]"
2,40828142,2024-04-12 00:14:34.844068+00:00,NaN,t,NaN,122043317385953537,864691132097585017,123448011605934094,864691132349699414,"[363618, 178376, 1240]","[373770, 158410, 12]","[364837, 178495, 1222]"
3,40887153,2024-04-12 00:14:37.031216+00:00,NaN,t,NaN,122254354831900849,864691132061572452,123448011605934094,864691132349699414,"[365364, 177676, 1078]","[373770, 158410, 12]","[365369, 177777, 1092]"
4,40978357,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,122324380716892373,864691132214831225,123448011605934094,864691132349699414,"[365874, 175602, 2464]","[373770, 158410, 12]","[364761, 176273, 2353]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1293,40854027,2024-04-12 00:14:35.561624+00:00,NaN,t,NaN,124012611363668277,864691131487124669,123448011605934094,864691132349699414,"[377900, 170506, 1125]","[373770, 158410, 12]","[377907, 170533, 1133]"
1294,40838412,2024-04-12 00:14:35.080948+00:00,NaN,t,NaN,123942036125515831,864691132277457428,123448011605934094,864691132349699414,"[377822, 169148, 511]","[373770, 158410, 12]","[376697, 169627, 234]"
1295,40894751,2024-04-12 00:14:37.286567+00:00,NaN,t,NaN,127108080595894442,864691132417967416,123448011605934094,864691132349699414,"[400574, 165132, 1803]","[373770, 158410, 12]","[370957, 183129, 2474]"
1296,40934384,2024-04-12 00:14:38.723502+00:00,NaN,t,NaN,124223648474071220,864691132119144326,123448011605934094,864691132349699414,"[379552, 170108, 334]","[373770, 158410, 12]","[378845, 170624, 280]"


In [138]:
client.materialize.synapse_query(post_ids=864691132349699414)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40869646,2024-04-12 00:14:36.782540+00:00,NaN,t,NaN,123942379588682010,864691132400996338,123448011605934094,864691132349699414,"[377628, 171754, 223]","[373770, 158410, 12]","[373974, 173811, 799]"
1,40978357,2024-04-12 00:14:40.110821+00:00,NaN,t,NaN,122324380716892373,864691132214831225,123448011605934094,864691132349699414,"[365874, 175602, 2464]","[373770, 158410, 12]","[364761, 176273, 2353]"
2,40828142,2024-04-12 00:14:34.844068+00:00,NaN,t,NaN,122043317385953537,864691132097585017,123448011605934094,864691132349699414,"[363618, 178376, 1240]","[373770, 158410, 12]","[364837, 178495, 1222]"
3,40960264,2024-04-12 00:14:39.879227+00:00,NaN,t,NaN,122043248733585629,864691131912892749,123448011605934094,864691132349699414,"[363756, 178168, 1305]","[373770, 158410, 12]","[363821, 178189, 1339]"
4,40887153,2024-04-12 00:14:37.031216+00:00,NaN,t,NaN,122254354831900849,864691132061572452,123448011605934094,864691132349699414,"[365364, 177676, 1078]","[373770, 158410, 12]","[365369, 177777, 1092]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1278,40957411,2024-04-12 00:14:39.566756+00:00,NaN,t,NaN,123943067387428985,864691132275754636,123448011605934094,864691132349699414,"[377733, 176968, 1282]","[373770, 158410, 12]","[378474, 176596, 1502]"
1279,40934384,2024-04-12 00:14:38.723502+00:00,NaN,t,NaN,124223648474071220,864691132119144326,123448011605934094,864691132349699414,"[379552, 170108, 334]","[373770, 158410, 12]","[378845, 170624, 280]"
1280,40838412,2024-04-12 00:14:35.080948+00:00,NaN,t,NaN,123942036125515831,864691132277457428,123448011605934094,864691132349699414,"[377822, 169148, 511]","[373770, 158410, 12]","[376697, 169627, 234]"
1281,40894751,2024-04-12 00:14:37.286567+00:00,NaN,t,NaN,127108080595894442,864691132417967416,123448011605934094,864691132349699414,"[400574, 165132, 1803]","[373770, 158410, 12]","[370957, 183129, 2474]"


In [140]:
bounding_box = [[373770, 158410, 12], [373780, 158420, 13]]
client.materialize.synapse_query(post_ids=864691132349699414, bounding_box=bounding_box)

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,40819798,2024-04-12 00:14:34.844068+00:00,NaN,t,NaN,123099949066879392,864691132147258441,123448011605934094,864691132349699414,"[371496, 186506, 3193]","[373770, 158410, 12]","[371900, 186987, 2966]"
1,40819918,2024-04-12 00:14:34.844068+00:00,NaN,t,NaN,123096925141467463,864691131166166970,123448011605934094,864691132349699414,"[371414, 164008, 2583]","[373770, 158410, 12]","[371434, 164062, 2594]"
2,40822400,2024-04-12 00:14:34.844068+00:00,NaN,t,NaN,122535486680990115,864691131803614149,123448011605934094,864691132349699414,"[367486, 175430, 1963]","[373770, 158410, 12]","[367604, 175470, 1972]"
3,40830335,2024-04-12 00:14:35.080948+00:00,NaN,t,NaN,123518723947496217,864691132132208949,123448011605934094,864691132349699414,"[374562, 160882, 84]","[373770, 158410, 12]","[374536, 160856, 102]"
4,40832563,2024-04-12 00:14:35.080948+00:00,NaN,t,NaN,121620693275115582,864691132325074179,123448011605934094,864691132349699414,"[360702, 175234, 2555]","[373770, 158410, 12]","[361034, 175657, 2240]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1278,40965944,2024-04-12 00:14:39.879227+00:00,NaN,t,NaN,122887536023437448,864691132031249548,123448011605934094,864691132349699414,"[369962, 177026, 998]","[373770, 158410, 12]","[369498, 176393, 1147]"
1279,40849535,2024-04-12 00:14:35.561624+00:00,NaN,t,NaN,121902168117608915,864691132110041215,123448011605934094,864691132349699414,"[362808, 175582, 2273]","[373770, 158410, 12]","[362571, 176066, 2291]"
1280,40873420,2024-04-12 00:14:36.782540+00:00,NaN,t,NaN,124645174147023046,864691132269116436,123448011605934094,864691132349699414,"[382914, 165014, 1052]","[373770, 158410, 12]","[376146, 171509, 1388]"
1281,40898622,2024-04-12 00:14:37.548445+00:00,NaN,t,NaN,121127974492701178,864691129131398651,123448011605934094,864691132349699414,"[357103, 174170, 2238]","[373770, 158410, 12]","[357089, 174133, 2260]"
